# 236605 Final Project

## KNN Demonstration 

In [1]:
import sklearn as sk
import matplotlib.pyplot as plt
import torchvision.transforms as tvtf
from ECG_multi_lead_dataloader import *
import transforms as tf
import torchvision
import torch
import knn_classifier as knn
import os

In [3]:
# Define the transforms that should be applied to each ECG record before returning it
tf_ds = tvtf.Compose([
    tf.ECG_tuple_transform(-1) # Reshape to 1D Tensor
])

In [4]:
root_dir = r'C:\Users\noam\Desktop\vadimDB'+'\\'

ECG_test=ECG_Multilead_Dataset(root_dir=root_dir,transform= tf_ds) # For KNN demo

Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Uploaded data, size of (41830, 2)


In [8]:
# Define how much data to load (only use a subset for speed)
num_train = 35000
num_test = 1000
batch_size = 10000

# Training dataset & loader
ds_train = tf.SubsetDataset(ECG_test, num_train)  #(train=True, transform=tf_ds)
dl_train = torch.utils.data.DataLoader(ds_train,batch_size= batch_size,shuffle=False)

# Test dataset & loader
ds_test = tf.SubsetDataset(ECG_test, num_test, offset= num_train)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size)

# Get all test data to predict in one go
test_iter = iter(dl_test)
x_test, y_test = test_iter.next()

In [13]:
knn_classifier = knn.KNNClassifier(k=10)
knn_classifier.train(dl_train)
y_pred = knn_classifier.predict(x_test)

# Calculate accuracy
accuracy = knn.accuracy(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 78.50%


## Digitized 12 Lead ECG classification

Clean up the environment from the previous exmple to free the memory (if necessary):

In [55]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


Imports and preperations, change root_dir here to the directory of the data pickles

In [1]:
import torch
import models
import transforms as tf
import matplotlib.pyplot as plt
from ECG_multi_lead_dataloader import *

root_dir = r'C:\Users\noam\Desktop\vadimDB'+'\\'

%load_ext autoreload
%autoreload 2

plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
ds = ECG_Multilead_Dataset(root_dir=root_dir)
    
# Define how much data to load (only use a subset for speed)
num_train = 35000
num_test = 1000
batch_size = 128

# Training dataset & loader
ds_train = tf.SubsetDataset(ds, num_train)  #(train=True, transform=tf_ds)
dl_train = torch.utils.data.DataLoader(ds_train,batch_size= batch_size,shuffle=False)

# Test dataset & loader
ds_test = tf.SubsetDataset(ds, num_test, offset= num_train)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size)

Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Loaded data with type of: <class 'list'>
Uploaded data, size of (41830, 2)


In [3]:
x, y = iter(dl_train).next()
x1, x2 = x
print('Long lead data of shape: ', x2.shape)
print('Short lead data of shape: ', x1.shape)
print('Labels of shape: ', y.shape)

Long lead data of shape:  torch.Size([128, 5000])
Short lead data of shape:  torch.Size([128, 12, 1250])
Labels of shape:  torch.Size([128])


In [17]:
short_hidden_channels = [16, 32, 64, 128, 256, 512]
long_hidden_channels  = [4, 8, 16, 32, 64, 128, 256, 512]
short_kernel_lengths = [5]*6
long_kernel_lengths = [5]*8
fc_hidden_dims = [128]
short_dropout = None
long_dropout = None
short_stride = 2
long_stride = 2
short_dilation = 1
long_dilation = 1
short_batch_norm = True
long_batch_norm = True
short_input_length = 1250
long_input_length = 5000
num_of_classes = 2

In [23]:
model = models.Ecg12LeadNet(short_hidden_channels, long_hidden_channels,
                 short_kernel_lengths, long_kernel_lengths,
                 fc_hidden_dims,
                 short_dropout, long_dropout,
                 short_stride, long_stride,
                 short_dilation, long_dilation,
                 short_batch_norm, long_batch_norm,
                 short_input_length, long_input_length,
                 num_of_classes).to(device)

In [24]:
x_try = (x1.to(device, dtype=torch.float), x2.unsqueeze(1).to(device, dtype=torch.float))
y_pred = model(x_try)
print(y_pred.shape)

torch.Size([128, 2])
